## Hastad's attack

We know that a message $m$ has been encrypted using RSA keys of the form $\langle e,N_i \rangle$,  $k$ times.  

Given that $k \geq e$, we can recover $m^e$ (and consecutively $m$) by applying the Chinese Remainder Theorem (CRT) underlied by the following isomorphism:


$ \mathbb{Z}/N_1N_2...N_k\mathbb{Z} \cong \mathbb{Z}/N_1\mathbb{Z} \times ... \times \mathbb{Z}/N_k\mathbb{Z}$

Note that we can assume that all N are coprime, since in case they shared a factor, we could recover $ p_i$ and $q_i$.

https://en.wikipedia.org/wiki/Chinese_remainder_theorem#Using_the_existence_construction


In [1]:
def bytes_to_long(bts):
    return int(bts.hex(), base=16)

def long_to_bytes(lng):
    return bytes.fromhex(hex(lng)[2:])

In [16]:

e = 3

Ns = [ random_prime(2**1024) * random_prime(2**1024) for i in range(e)]



m = bytes_to_long(b"Well hidden message!!!! Lorem ipsum \
  dolor sit amet, consectetur adipiscing elit, \
  sed do eiusmod tempor incididunt ut labore ")

Cts = [pow(m, e, n) for n in Ns]


Reference crt implementations:  
https://github.com/sympy/sympy/blob/master/sympy/polys/galoistools.py#L12  
https://cp-algorithms.com/algebra/chinese-remainder-theorem.html  
https://wiki.math.ntnu.no/_media/tma4155/2010h/euclid.pdf  


Working mod $a$

In [ ]:
def xgcd(a, b):
    """
    Implementation of the Extended Euclidean Algorithm
    a, b -> integers
    """
    
    a1, b1 = a, b
    x0, x1 = 1, 0
    y0, y1 = 0, 1
    
    while b1 != 0:
    
        q = a1 // b1
        x0, x1 = x1, x0 - q * x1
        y0, y1 = y1, y0 - q * y1
        a1, b1 = b1, a1 - q * b1
    
    return (x0, y0, a1)
    
    
    


def crt(r, m):
    """
    Implementation of the Chinese Remainder Theorem
    r -> list of residues
    m -> list of modulos
    """
    assert len(m) == len(r)
    
    
    m1, r1 = m[0], r[0]
    
    for m2, r2 in zip(m[1:], r[1:]):
        #note that the moduli are assumed to be coprime
        a1, a2, _ = xgcd(m1, m2)
        
        
        """
        mod m1, everything except r1 cancels out since:
        a1*m1 + a2*m2 = 1
        SImilarly, mod m2 everything except r2 cancels out proving that
        this is a solution for (ri, r)
        """
        
        r1 = (r1 * a2 * m2 + r2 * a1 * m1) % (m1 * m2)
        m1 *= m2
        
    return (r1, m1)
        
        

Notice that $a_1m_1 + a_2m_2 = 1$

$ \langle r_1,m_1 \rangle$ is indeed a recursively produced solution since:  
$r_1a_2m_2 + r_2a_1m_1 \equiv r_1(1 - a_1m_1) + r_2a_1m_1 \equiv r_1 \mod m_1 $   

Similarly,   $ r_1a_2m_2 + r_2a_1m_1 \equiv r_2 \mod m_2 $

Having implemented CRT we can now recover $m$:

In [ ]:
m_e, _ = crt(Cts, Ns)

m = m_e.nth_root(3)

print(long_to_bytes(m))
